---
title: Migrating from MultiPromptChain
---

The [`MultiPromptChain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html) routed an input query to one of multiple LLMChains-- that is, given an input query, it used a LLM to select from a list of prompts, formatted the query into the prompt, and generated a response.

`MultiPromptChain` does not support common [chat model](/docs/concepts/#chat-models) features, such as message roles and [tool calling](/docs/concepts/#functiontool-calling).

Some advantages of switching to the LCEL implementation are:

- Supports chat prompt templates, including messages with `system` and other roles;
- Supports the use of tool calling for the routing step;
- Support for runnable methods like streaming and async operations.

Now let's look at them side-by-side. Note that for this guide we will `langchain-openai >= 0.1.20`

In [ ]:
%pip install -qU langchain-core langchain-openai

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

import { ColumnContainer, Column } from "@theme/Columns";

<ColumnContainer>

<Column>

#### Legacy


In [1]:
from langchain.chains.router.multi_prompt import MultiPromptChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

prompt_1_template = """
You are an expert on animals. Please answer the beloq query:

{input}
"""

prompt_2_template = """
You are an expert on vegetables. Please answer the beloq query:

{input}
"""

prompt_infos = [
    {
        "name": "animals",
        "description": "prompt for an animal expert",
        "prompt_template": prompt_1_template,
    },
    {
        "name": "vegetables",
        "description": "prompt for a vegetable expert",
        "prompt_template": prompt_2_template,
    },
]

chain = MultiPromptChain.from_prompts(llm, prompt_infos)

In [2]:
chain.invoke({"input": "What color are carrots?"})

{'input': 'What colors are carrots?',
 'text': 'Carrots come in various colors such as orange, red, yellow, purple, and white. The most common color of carrots is orange, but there are also heirloom varieties that come in different colors.'}


</Column>

<Column>

#### LCEL


In [3]:
from operator import itemgetter
from typing import Literal

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Define the prompts we will route to
prompt_1 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert on animals."),
        ("human", "{input}"),
    ]
)
prompt_2 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert on vegetables."),
        ("human", "{input}"),
    ]
)

# Construct the chains we will route to. These format the input query
# into the respective prompt, run it through a chat model, and cast
# the result to a string.
chain_1 = prompt_1 | llm | StrOutputParser()
chain_2 = prompt_2 | llm | StrOutputParser()


# Next: define the chain that selects which branch to route to.
# Here we will take advantage of tool-calling features to force
# the output to select one of two desired branches.
route_system = "Route the user's query to either the animal or vegetable expert."
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{input}"),
    ]
)


# Define schema for output:
class RouteQuery(TypedDict):
    """Route query to destination expert."""

    destination: Literal["animal", "vegetable"]


route_chain = (
    route_prompt | llm.with_structured_output(RouteQuery) | itemgetter("destination")
)


# Finally, assemble the multi-prompt chain. This is a sequence of two steps:
# 1) Select "animal" or "vegetable" via the route_chain, and collect the answer
# alongside the input query.
# 2) Use RunnableBranch to route the input query to chain_1 or chain_2, based on the
# selection.
chain = {
    "destination": route_chain,  # "animal" or "vegetable"
    "input": RunnablePassthrough(),  # pass through input query
} | RunnableBranch(
    (lambda x: x["destination"] == "animal", chain_1),  # if animal, chain_1
    chain_2,  # otherwise, chain_2
)

In [4]:
chain.invoke({"input": "what color are carrots"})

'Carrots are typically orange in color, but they can also be found in other colors such as purple, red, yellow, and white, depending on the variety.'


</Column>
</ColumnContainer>

## Overview:

- Under the hood, `MultiPromptChain` routes the query by instructing the LLM to generate JSON-formatted text, and parses out the intended destination. It takes a registry of string prompt templates as input.
- The LCEL implementation, implemented above via lower-level primitives, uses tool-calling to route to arbitrary chains. In this example, the chains include chat model templates and chat models.

## Next steps

See [this tutorial](/docs/tutorials/llm_chain) for more detail on building with prompt templates, LLMs, and output parsers.

Check out the [LCEL conceptual docs](/docs/concepts/#langchain-expression-language-lcel) for more background information.